In [1]:
import fs from "node:fs";
import { dirname } from "node:path";
import { exec } from "node:child_process";

import * as Plot from "npm:@observablehq/plot";
import * as d3 from "npm:d3";
import { JSDOM } from "npm:jsdom";
import * as pl from "npm:nodejs-polars";
import { readCSV } from "npm:nodejs-polars";
import puppeteer from "npm:puppeteer";


In [2]:
function uniformSample<D extends pl.DataFrame>(df: D, targetSize: number = 1000): D {
  if (df.height <= targetSize) return df;

  const sampledColumns = df.columns.map(() => []);
  const ratio = df.height / targetSize;

  for (let i = 0; i < targetSize; i++) {
    const index = Math.round(i * ratio);
    const row = df.row(index);
    for (let j = 0; j < df.width; j++) sampledColumns[j].push(row[j]);
  }

  return pl.DataFrame(Object.fromEntries(sampledColumns.map((col, i) => [df.columns[i], col])));
}


In [3]:
async function savePNG(
  plot: Plot.Plot,
  path: string,
  options: {
    /** Padding around the plot in pixels */
    padding?: number;
  } = {}
) {
  const { padding = 0 } = options;

  const browser = await puppeteer.launch({
    headless: true,
    args: ["--no-sandbox", "--disable-setuid-sandbox"],
  });

  const page = await browser.newPage();

  await page.setViewport({
    // Simulate 4K resolution
    width: 3840,
    height: 2160,
    deviceScaleFactor: 2, // High DPI for better quality
  });

  const htmlContent =
    "<!DOCTYPE html>\n" +
    '<html lang="en">\n' +
    "  <head>\n" +
    '    <meta charset="UTF-8" />\n' +
    '    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n' +
    "    <style>\n" +
    "      body { margin: 0; }\n" +
    "      h2 { margin-top: 0; }\n" +
    "    </style>\n" +
    "  </head>\n" +
    "  <body>\n    " +
    plot.outerHTML.replace("max-width: initial;", `width: fit-content; padding: ${padding}px;`) +
    "\n  </body>\n" +
    "</html>";
  await page.setContent(htmlContent);

  // Capture the figure element
  const figureElement = (await page.$("figure")) || (await page.$("svg"));

  if (!figureElement) throw new Error("Figure element not found");

  const boundingBox = await figureElement.boundingBox();

  if (!boundingBox) throw new Error("Could not get figure bounding box");

  if (dirname(path) !== ".") await fs.promises.mkdir(dirname(path), { recursive: true });

  // Screenshot
  await figureElement.screenshot({
    path,
    type: "png",
    omitBackground: false, // Include background
  });

  await browser.close();
}

async function savePDF(
  plot: Plot.Plot,
  path: string,
  options: {
    /** Padding around the plot in pixels */
    padding?: number;
    /** Paper format */
    format?: "A4" | "A3" | "A5" | "Letter" | "Legal" | "Tabloid";
    printBackground?: boolean;
  } = {}
) {
  const { padding = 0, format = "A4", printBackground = true } = options;

  const browser = await puppeteer.launch({
    headless: true,
    args: ["--no-sandbox", "--disable-setuid-sandbox"],
  });

  const page = await browser.newPage();

  const htmlContent =
    "<!DOCTYPE html>\n" +
    '<html lang="en">\n' +
    "  <head>\n" +
    '    <meta charset="UTF-8" />\n' +
    '    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n' +
    "    <style>\n" +
    "      body { margin: 0; font-family: 'Times New Roman', serif; }\n" +
    "      h2 { margin-top: 0; }\n" +
    "      figure { margin: 0; }\n" +
    "    </style>\n" +
    "  </head>\n" +
    "  <body>\n    " +
    plot.outerHTML.replace("max-width: initial;", `width: fit-content; padding: ${padding}px;`) +
    "\n  </body>\n" +
    "</html>";
  await page.setContent(htmlContent);

  await page.pdf({
    path,
    format,
    printBackground,
    preferCSSPageSize: true,
  });

  if (dirname(path) !== ".") await fs.promises.mkdir(dirname(path), { recursive: true });

  await browser.close();
}

async function saveEPS(
  plot: Plot.Plot,
  path: string,
  options: {
    /** Padding around the plot in pixels */
    padding?: number;
    printBackground?: boolean;
  } = {}
) {
  const { padding = 0, printBackground = true } = options;

  const tempFilePath = path.replace(/\.eps$/, "") + "-" + Date.now() + ".pdf";

  const browser = await puppeteer.launch({
    headless: true,
    args: ["--no-sandbox", "--disable-setuid-sandbox"],
  });

  const page = await browser.newPage();

  const htmlContent =
    "<!DOCTYPE html>\n" +
    '<html lang="en">\n' +
    "  <head>\n" +
    '    <meta charset="UTF-8" />\n' +
    '    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n' +
    "    <style>\n" +
    "      body { margin: 0; font-family: 'Times New Roman', serif; }\n" +
    "      h2 { margin-top: 0; }\n" +
    "      figure { margin: 0; }\n" +
    "    </style>\n" +
    "  </head>\n" +
    "  <body>\n    " +
    plot.outerHTML.replace("max-width: initial;", `width: fit-content; padding: ${padding}px;`) +
    "\n  </body>\n" +
    "</html>";
  await page.setContent(htmlContent);

  const figure = (await page.$("figure")) || (await page.$("svg"));
  if (!figure) throw new Error("Figure element not found");
  const box = await figure.boundingBox();
  if (!box) throw new Error("Could not compute bounding box");

  if (dirname(path) !== ".") await fs.promises.mkdir(dirname(path), { recursive: true });

  await page.pdf({
    path: tempFilePath,
    printBackground,
    width: `${Math.ceil(box.width)}px`,
    height: `${Math.ceil(box.height)}px`,
  });

  await browser.close();

  try {
    await new Promise((resolve, reject) => {
      exec(`inkscape "${tempFilePath}" --export-filename="${path}"`, (error) => {
        if (error) reject(error);
        else resolve(null);
      });
    });
  } catch (error) {
    console.error("inkscape not found. Please install Inkscape");
    console.error("Ubuntu/Debian: sudo apt install inkscape");
    console.error("macOS: brew install inkscape");
  }

  await fs.promises.rm(tempFilePath, { force: true });
}


In [4]:
const data = [0, 0.05, 500].flatMap((alpha) => {
  let df = readCSV(
    fs.readFileSync(
      `../output/synthetic_${alpha === 0 ? "cms" : "alpha" + alpha}.trace.csv`,
      "utf-8"
    )
  ).select(pl.col("objective").as("dcg"));
  df = uniformSample(df, 400);
  return df.toRecords().map((d, i) => ({
    index: i,
    alpha:
      alpha === 0
        ? "CMS"
        : alpha === 0.05
        ? "Slow decay (α=" + alpha + ")"
        : alpha === 500
        ? "Fast decay (α=" + alpha + ")"
        : `α=${alpha}`,
    ...d,
  }));
});
const options = {
  width: 1000,
  color: { legend: true },
  x: {
    label: "Time",
    labelAnchor: "center",
  },
  y: {
    label: "DCG (×10³)",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "index",
      y: (d) => d.dcg / 1000,
      stroke: "alpha",
      strokeWidth: 2.5,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 72,
  marginBottom: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 64 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/dcg_synth_intro.eps");
Plot.plot({
  ...options,
  title: "Synthetic Online E-Commerce Product Ranking - DCG Timeline",
});


Synthetic Online E-Commerce Product Ranking - DCG Timeline CMS Fast decay (α=500) Slow decay (α=0.05) 0.5 1.0 1.5 DCG (×10³) 0 50 100 150 200 250 300 350 Time → <path stroke="#4269d0" d="M40,82.662L42.356,242.116L44.712,125.209L47.068,265.478L49.424,126.64L51.779,133.421L54.135,139.262L56.491,156.443L58.847,149.541L61.203,141.343L63.559,138.627L65.915,129.387L68.271,140.501L70.627,149.377L72.982,142.27L75.338,146.469L77.694,147.305L80.05,143.451L82.406,159.88L84.762,161.299L87.118,155.701L89.474,148.627L91.83,150.644L94.185,141.342L96.541,139.408L98.897,304.845L101.253,295.962L103.609,290.769L105.965,306.37L108.321,316.476L110.677,315.348L113.033,312.901L115.388,312.601L117.744,292.093L120.1,287.885L122.456,293.784L124.812,289.661L127.168,285.359L129.524,271.394L131.88,299.009L134.236,311.105L136.591,301.243L138.947,321.002L141.303,317.8L143.659,301.629L146.015,301.008L148.371,300.694L150.727,297.457L153.083,319.499L155.439,320.772L157.794,316.621L160.15,324.317L162.506,316.804L164.862,318.436L167.218,306.696L169.574,307.586L171.93,318.635L174.286,311.088L176.642,316.905L178.997,318.06L181.353,320.643L183.709,310.759L186.065,308.451L188.421,337.512L190.777,336.252L193.133,335.412L195.489,318.506L197.845,315.647L200.201,313.101L202.556,314.009L204.912,310.045L207.268,314.556L209.624,322.577L211.98,323.472L214.336,328.182L216.692,343.134L219.048,342.325L221.404,330.026L223.759,325.936L226.115,328.905L228.471,329.247L230.827,329.455L233.183,326.599L235.539,329.708L237.895,328.85L240.251,330.348L242.607,321.275L244.962,322.618L247.318,344.615L249.674,346.433L252.03,353.747L254.386,354.493L256.742,336.961L259.098,334.597L261.454,333.562L263.81,334.969L266.165,334.912L268.521,332.897L270.877,354.632L273.233,358.015L275.589,355.523L277.945,340.082L280.301,335.824L282.657,335.807L285.013,338.986L287.368,342.088L289.724,339.325L292.08,337.927L294.436,336.253L296.792,325.947L299.148,324.711L301.504,321.892L303.86,324.833L306.216,322.472L308.571,317.598L310.927,315.572L313.283,314.039L315.639,317.323L317.995,314.971L320.351,266.7L322.707,368.581L325.063,348.097L327.419,368.026L329.774,342.376L332.13,368.32L334.486,368.906L336.842,368.832L339.198,364.412L341.554,369.554L343.91,364.922L346.266,369.992L348.622,367.345L350.977,361.863L353.333,368.91L355.689,369.975L358.045,369.237L360.401,345.248L362.757,363.674L365.113,369.675L367.469,369.242L369.825,369.563L372.18,369.466L374.536,369.324L376.892,369.507L379.248,369.311L381.604,343.347L383.96,366.864L386.316,368.485L388.672,369.12L391.028,367.149L393.383,343.353L395.739,369.738L398.095,357.534L400.451,367.42L402.807,365.966L405.163,368.03L407.519,368.102L409.875,369.589L412.231,368.599L414.586,368.83L416.942,370L419.298,369.603L421.654,356.706L424.01,368.386L426.366,368.988L428.722,368.67L431.078,368.802L433.434,367.21L435.789,364.676L438.145,364.473L440.501,368.737L442.857,330.637L445.213,369.7L447.569,369.685L449.925,369.701L452.281,369.303L454.637,357.199L456.992,365.852L459.348,367.888L461.704,364.24L464.06,356.285L466.416,343.399L468.772,369.555L471.128,362.436L473.484,366.916L475.84,369.996L478.195,353.915L480.551,365.837L482.907,369.588L485.263,367.683L487.619,367.909L489.975,361.955L492.331,366.352L494.687,368.142L497.043,369.082L499.398,365.942L501.754,368.473L504.11,369.114L506.466,369.693L508.822,368.728L511.178,367.86L513.534,369.611L515.89,369.541L518.246,368.781L520.602,364.848L522.957,369.125L525.313,358.076L527.669,364.15L530.025,368.821L532.381,333.396L534.737,361.273L537.093,369.467L539.449,368.573L541.805,368.99L544.16,369.196L546.516,369.446L548.872,368.874L551.228,368.187L553.584,368.537L555.94,359.516L558.296,333.78L560.652,367.635L563.008,324.326L565.363,369.49L567.719,369.402L570.075,368.9L572.431,369.515L574.787,364.573L577.143,368.892L579.499,368.08L581.855,368.11L584.211,368.453L586.566,360.809L588.922,368.778L591.278,358.579L593.634,367.975L595.99,363.033L598.346,366.631L600.702,369.684L603.058,369.694L605.414,368.666L607.769,367.233L610.125,362.825L612.481,302.6

In [5]:
const df = readCSV(fs.readFileSync("../output/meta_large.csv", "utf-8"))
  .filter(pl.col("type").eq(pl.lit("miss_ratio")))
  .select(
    pl.col("alpha"),
    pl.col("W-TinyLFU_CMS").as("CMS"),
    pl.col("W-TinyLFU_ADA").as("Ada"),
    pl.col("W-TinyLFU_EVO_PRUNING_ONLY").as("Evo-PO"),
    pl.col("W-TinyLFU_EVO (Ia=1000)").as("Evo1"),
    pl.col("W-TinyLFU_EVO (Ia=10000)").as("Evo2"),
    pl.col("W-TinyLFU_EVO (Ia=100000)").as("Evo3")
  );
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, missRatio]) => ({ alpha: record.alpha, baseline, missRatio }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -40,
  },
  y: {
    label: "Miss Ratio",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.089, 0.154],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 64 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/miss_ratio_meta_large.eps");
Plot.plot({
  ...options,
  title: "Meta KV (Large) - Miss Ratio V.S. Alpha",
});


Meta KV (Large) - Miss Ratio V.S. Alpha Ada CMS Evo-PO Evo1 Evo2 Evo3 0.09 0.10 0.11 0.12 0.13 0.14 0.15 Miss Ratio 0.01 0.02 0.05 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [6]:
const df = readCSV(fs.readFileSync("../output/meta_small.csv", "utf-8"))
  .filter(pl.col("type").eq(pl.lit("miss_ratio")))
  .select(
    pl.col("alpha"),
    pl.col("W-TinyLFU_CMS").as("CMS"),
    pl.col("W-TinyLFU_ADA").as("Ada"),
    pl.col("W-TinyLFU_EVO_PRUNING_ONLY").as("Evo-PO"),
    pl.col("W-TinyLFU_EVO (Ia=1000)").as("Evo1"),
    pl.col("W-TinyLFU_EVO (Ia=10000)").as("Evo2"),
    pl.col("W-TinyLFU_EVO (Ia=100000)").as("Evo3")
  );
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, missRatio]) => ({ alpha: record.alpha, baseline, missRatio }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -40,
  },
  y: {
    label: "Miss Ratio",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.158, 0.275],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 64 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/miss_ratio_meta_small.eps");
Plot.plot({
  ...options,
  title: "Meta KV Trace (Small) - Miss Ratio V.S. Alpha",
});


Meta KV Trace (Small) - Miss Ratio V.S. Alpha Ada CMS Evo-PO Evo1 Evo2 Evo3 0.16 0.18 0.20 0.22 0.24 0.26 Miss Ratio 0.01 0.02 0.05 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [7]:
const df = readCSV(fs.readFileSync("../output/msr_large.csv", "utf-8"))
  .filter(pl.col("type").eq(pl.lit("miss_ratio")))
  .select(
    pl.col("alpha"),
    pl.col("W-TinyLFU_CMS").as("CMS"),
    pl.col("W-TinyLFU_ADA").as("Ada"),
    pl.col("W-TinyLFU_EVO_PRUNING_ONLY").as("Evo-PO"),
    pl.col("W-TinyLFU_EVO (Ia=1000)").as("Evo1"),
    pl.col("W-TinyLFU_EVO (Ia=10000)").as("Evo2"),
    pl.col("W-TinyLFU_EVO (Ia=100000)").as("Evo3")
  );
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, missRatio]) => ({ alpha: record.alpha, baseline, missRatio }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -40,
  },
  y: {
    label: "Miss Ratio",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.0015, 0.0315],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 108,
  marginBottom: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 64 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/miss_ratio_msr_large.eps");
Plot.plot({
  ...options,
  title: "MSR Trace (Large) - Miss Ratio V.S. Alpha",
});


MSR Trace (Large) - Miss Ratio V.S. Alpha Ada CMS Evo-PO Evo1 Evo2 Evo3 0.005 0.010 0.015 0.020 0.025 0.030 Miss Ratio 0.01 0.02 0.05 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [8]:
const df = readCSV(fs.readFileSync("../output/msr_small.csv", "utf-8"))
  .filter(pl.col("type").eq(pl.lit("miss_ratio")))
  .select(
    pl.col("alpha"),
    pl.col("W-TinyLFU_CMS").as("CMS"),
    pl.col("W-TinyLFU_ADA").as("Ada"),
    pl.col("W-TinyLFU_EVO_PRUNING_ONLY").as("Evo-PO"),
    pl.col("W-TinyLFU_EVO (Ia=1000)").as("Evo1"),
    pl.col("W-TinyLFU_EVO (Ia=10000)").as("Evo2"),
    pl.col("W-TinyLFU_EVO (Ia=100000)").as("Evo3")
  );
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, missRatio]) => ({ alpha: record.alpha, baseline, missRatio }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -40,
  },
  y: {
    label: "Miss Ratio",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.405, 0.525],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 64 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/miss_ratio_msr_small.eps");
Plot.plot({
  ...options,
  title: "MSR Trace (Small) - Miss Ratio V.S. Alpha",
});


MSR Trace (Small) - Miss Ratio V.S. Alpha Ada CMS Evo-PO Evo1 Evo2 Evo3 0.42 0.44 0.46 0.48 0.50 0.52 Miss Ratio 0.01 0.02 0.05 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [9]:
const df = readCSV(fs.readFileSync("../output/hm_large.csv", "utf-8"))
  .filter(pl.col("type").eq(pl.lit("dcg")))
  .select(
    pl.col("alpha"),
    pl.col("CMS"),
    pl.col("ADA").as("Ada"),
    pl.col("EVO_PRUNING_ONLY").as("Evo-PO"),
    pl.col("EVO (Ia=1000)").as("Evo1"),
    pl.col("EVO (Ia=10000)").as("Evo2"),
    pl.col("EVO (Ia=100000)").as("Evo3")
  );
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, dcg]) => ({ alpha: record.alpha, baseline, dcg: dcg / 1_000_000 }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -45,
  },
  y: {
    label: "DCG (×10⁶)",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    // domain: [0.0135, 0.0365],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "dcg",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "dcg",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 80,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 72 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/dcg_hm_large.eps");
Plot.plot({
  ...options,
  title: "H&M Online Product Ranking (K=1000) - DCG V.S. Alpha",
});


H&M Online Product Ranking (K=1000) - DCG V.S. Alpha Ada CMS Evo-PO Evo1 Evo2 Evo3 0.5 0.6 0.7 0.8 0.9 1.0 DCG (×10⁶) 0.01 0.02 0.05 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [10]:
const df = readCSV(fs.readFileSync("../output/hm_small.csv", "utf-8"))
  .filter(pl.col("type").eq(pl.lit("dcg")))
  .select(
    pl.col("alpha"),
    pl.col("CMS"),
    pl.col("ADA").as("Ada"),
    pl.col("EVO_PRUNING_ONLY").as("Evo-PO"),
    pl.col("EVO (Ia=1000)").as("Evo1"),
    pl.col("EVO (Ia=10000)").as("Evo2"),
    pl.col("EVO (Ia=100000)").as("Evo3")
  );
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, dcg]) => ({ alpha: record.alpha, baseline, dcg: dcg / 1_000_000 }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -45,
  },
  y: {
    label: "DCG (×10⁶)",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    // domain: [0.0033, 0.0145],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "dcg",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "dcg",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 80,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 72 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/dcg_hm_small.eps");
Plot.plot({
  ...options,
  title: "H&M Online Product Ranking (K=100) - DCG V.S. Alpha",
});


H&M Online Product Ranking (K=100) - DCG V.S. Alpha Ada CMS Evo-PO Evo1 Evo2 Evo3 0.15 0.20 0.25 0.30 0.35 0.40 DCG (×10⁶) 0.01 0.02 0.05 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [11]:
const df = readCSV(fs.readFileSync("../output/synthetic_large.csv", "utf-8"))
  .filter(pl.col("type").eq(pl.lit("dcg")))
  .select(
    pl.col("alpha"),
    pl.col("CMS"),
    pl.col("ADA").as("Ada"),
    pl.col("EVO_PRUNING_ONLY").as("Evo-PO"),
    pl.col("EVO (Ia=1000)").as("Evo1"),
    pl.col("EVO (Ia=10000)").as("Evo2"),
    pl.col("EVO (Ia=100000)").as("Evo3")
  );
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, dcg]) => ({ alpha: record.alpha, baseline, dcg: dcg / 1_000_000 }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -45,
  },
  y: {
    label: "DCG (×10⁶)",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    // domain: [0.0135, 0.0365],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "dcg",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "dcg",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 80,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 72 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/dcg_synth_large.eps");
Plot.plot({
  ...options,
  title: "Synthetic E-Commerce Dataset Online Product Ranking (K=1000) - DCG V.S. Alpha",
});


Synthetic E-Commerce Dataset Online Product Ranking (K=1000) - DCG V.S. Alpha Ada CMS Evo-PO Evo1 Evo2 Evo3 0.5 1.0 1.5 2.0 2.5 DCG (×10⁶) 0.01 0.02 0.05 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [12]:
const df = readCSV(fs.readFileSync("../output/synthetic_small.csv", "utf-8"))
  .filter(pl.col("type").eq(pl.lit("dcg")))
  .select(
    pl.col("alpha"),
    pl.col("CMS"),
    pl.col("ADA").as("Ada"),
    pl.col("EVO_PRUNING_ONLY").as("Evo-PO"),
    pl.col("EVO (Ia=1000)").as("Evo1"),
    pl.col("EVO (Ia=10000)").as("Evo2"),
    pl.col("EVO (Ia=100000)").as("Evo3")
  );
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, dcg]) => ({ alpha: record.alpha, baseline, dcg: dcg / 1_000_000 }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -45,
  },
  y: {
    label: "DCG (×10⁶)",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    // domain: [0.0033, 0.0145],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "dcg",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "dcg",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 80,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 72 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/dcg_synth_small.eps");
Plot.plot({
  ...options,
  title: "Synthetic E-Commerce Dataset Online Product Ranking (K=100) - DCG V.S. Alpha",
});


Synthetic E-Commerce Dataset Online Product Ranking (K=100) - DCG V.S. Alpha Ada CMS Evo-PO Evo1 Evo2 Evo3 0.5 1.0 1.5 DCG (×10⁶) 0.01 0.02 0.05 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [13]:
const df = readCSV(fs.readFileSync("../output/msr_small.csv", "utf-8"))
  .filter(pl.col("type").eq(pl.lit("estimate_avg_time_s")))
  .select(
    pl.col("alpha"),
    pl.col("W-TinyLFU_CMS").as("CMS"),
    pl.col("W-TinyLFU_ADA").as("Ada"),
    pl.col("W-TinyLFU_EVO_PRUNING_ONLY").as("Evo-PO"),
    pl.col("W-TinyLFU_EVO (Ia=1000)").as("Evo1"),
    pl.col("W-TinyLFU_EVO (Ia=10000)").as("Evo2"),
    pl.col("W-TinyLFU_EVO (Ia=100000)").as("Evo3")
  );
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, time]) => ({
      alpha: record.alpha,
      baseline,
      throughput: 1.0 / time / 1_000_000,
    }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -45,
  },
  y: {
    label: "Query Throughput (Mops/s)",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0, 23],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "throughput",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "throughput",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 76,
  marginBottom: 80,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 72 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/query_throughput_msr.eps");
Plot.plot({
  ...options,
  title: "Query Throughput on MSR",
});


Query Throughput on MSR Ada CMS Evo-PO Evo1 Evo2 Evo3 0 5 10 15 20 Query Throughput (Mops/s) 0.01 0.02 0.05 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [14]:
const df = readCSV(fs.readFileSync("../output/msr_small.csv", "utf-8"))
  .filter(pl.col("type").eq(pl.lit("update_avg_time_s")))
  .select(
    pl.col("alpha"),
    pl.col("W-TinyLFU_CMS").as("CMS"),
    pl.col("W-TinyLFU_ADA").as("Ada"),
    pl.col("W-TinyLFU_EVO_PRUNING_ONLY").as("Evo-PO"),
    pl.col("W-TinyLFU_EVO (Ia=1000)").as("Evo1"),
    pl.col("W-TinyLFU_EVO (Ia=10000)").as("Evo2"),
    pl.col("W-TinyLFU_EVO (Ia=100000)").as("Evo3")
  );
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, time]) => ({
      alpha: record.alpha,
      baseline,
      throughput: 1.0 / time / 1_000_000,
    }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -45,
  },
  y: {
    label: "Query Throughput (Mops/s)",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0, 23],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "throughput",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "throughput",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 76,
  marginBottom: 80,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 72 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/update_throughput_msr.eps");
Plot.plot({
  ...options,
  title: "Update Throughput on MSR",
});


Update Throughput on MSR Ada CMS Evo-PO Evo1 Evo2 Evo3 0 5 10 15 20 Query Throughput (Mops/s) 0.01 0.02 0.05 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [15]:
const df = readCSV(fs.readFileSync("../output/hm_small.csv", "utf-8"))
  .filter(pl.col("type").eq(pl.lit("estimate_avg_time_s")))
  .select(
    pl.col("alpha"),
    pl.col("CMS"),
    pl.col("ADA").as("Ada"),
    pl.col("EVO_PRUNING_ONLY").as("Evo-PO"),
    pl.col("EVO (Ia=1000)").as("Evo1"),
    pl.col("EVO (Ia=10000)").as("Evo2"),
    pl.col("EVO (Ia=100000)").as("Evo3")
  );
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, time]) => ({
      alpha: record.alpha,
      baseline,
      throughput: 1.0 / time / 1_000_000,
    }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -45,
  },
  y: {
    label: "Query Throughput (Mops/s)",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0, 23],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "throughput",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "throughput",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 76,
  marginBottom: 80,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 72 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/query_throughput_hm.eps");
Plot.plot({
  ...options,
  title: "Query Throughput on H&M",
});


Query Throughput on H&M Ada CMS Evo-PO Evo1 Evo2 Evo3 0 5 10 15 20 Query Throughput (Mops/s) 0.01 0.02 0.05 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [16]:
const df = readCSV(fs.readFileSync("../output/hm_small.csv", "utf-8"))
  .filter(pl.col("type").eq(pl.lit("update_avg_time_s")))
  .select(
    pl.col("alpha"),
    pl.col("CMS"),
    pl.col("ADA").as("Ada"),
    pl.col("EVO_PRUNING_ONLY").as("Evo-PO"),
    pl.col("EVO (Ia=1000)").as("Evo1"),
    pl.col("EVO (Ia=10000)").as("Evo2"),
    pl.col("EVO (Ia=100000)").as("Evo3")
  );
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, time]) => ({
      alpha: record.alpha,
      baseline,
      throughput: 1.0 / time / 1_000_000,
    }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -45,
  },
  y: {
    label: "Query Throughput (Mops/s)",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0, 23],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "throughput",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "throughput",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 76,
  marginBottom: 80,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 72 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/update_throughput_hm.eps");
Plot.plot({
  ...options,
  title: "Update Throughput on H&M",
});


Update Throughput on H&M Ada CMS Evo-PO Evo1 Evo2 Evo3 0 5 10 15 20 Query Throughput (Mops/s) 0.01 0.02 0.05 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [17]:
const df = readCSV(fs.readFileSync("../output/meta.trace.csv", "utf-8")).select(
  pl.col("objective").as("hitRate"),
  pl.col("parameter").as("alpha")
);
console.log("Trace length:", df.height);
console.log(df.describe().toString());
const data = uniformSample(df, 1000)
  .toRecords()
  .map((d, i) => ({ index: i, ...d }));
const y2 = d3.scaleLog(
  d3.extent(data, (d) => d.alpha),
  d3.extent(data, (d) => 1 - d.hitRate)
);
const options = {
  x: { label: "Time", labelAnchor: "center" },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.axisY({
      axis: "left",
      color: "hsl(207, 45%, 30%)",
      label: "Miss Ratio",
      labelAnchor: "center",
      labelArrow: false,
      ticks: 5,
    }),
    Plot.axisY(y2.ticks(), {
      anchor: "right",
      color: "hsl(0, 53%, 40%)",
      label: "Alpha",
      labelAnchor: "center",
      labelArrow: false,
      y: y2,
      tickFormat: (d) => {
        let s = y2.tickFormat()(d);
        if (s === "200m") s = "0.2";
        return s;
      },
    }),
    Plot.lineY(data, { x: "index", y: (d) => 1 - d.hitRate, stroke: "steelblue" }),
    Plot.dotY(
      data,
      Plot.mapY((D) => D.map(y2), { x: "index", y: (d) => d.alpha, r: 1, stroke: "indianred" })
    ),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 64,
  marginRight: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 56 },
});
d3.select(plot).select("div").style("font-size", "20px");
await saveEPS(plot, "../output/figures/adaptation_timeline_meta.eps");
Plot.plot({
  ...options,
  title: "Meta KV Trace - Adaptation Timeline",
});


Trace length: 34030


shape: (5, 3)
┌──────────┬──────────┬────────────┐
│ describe ┆ hitRate  ┆ alpha      │
│ ---      ┆ ---      ┆ ---        │
│ str      ┆ f64      ┆ f64        │
╞══════════╪══════════╪════════════╡
│ mean     ┆ 0.830781 ┆ 44.222889  │
│ std      ┆ 0.053568 ┆ 118.785338 │
│ min      ┆ 0.6419   ┆ 0.01       │
│ max      ┆ 1.0      ┆ 1000.0     │
│ median   ┆ 0.8182   ┆ 4.229243   │
└──────────┴──────────┴────────────┘


Meta KV Trace - Adaptation Timeline 0 200 400 600 800 Time → 0.00 0.05 0.10 0.15 0.20 0.25 Miss Ratio 20m 100m 0.2 1 2 10 20 100 200 1k Alpha <path d="M40,228.992L40.561,98.582L41.121,211.156L41.682,261.821L42.242,20L42.803,62.005L43.363,96.902L43.924,51.019L44.484,69.114L45.045,58.257L45.606,24.265L46.166,93.154L46.727,72.862L47.287,144.982L47.848,70.535L48.408,331.226L48.969,179.49L49.53,129.472L50.09,41.196L50.651,102.718L51.211,103.881L51.772,62.393L52.332,129.472L52.893,172.511L53.453,114.35L54.014,94.963L54.575,75.835L55.135,70.406L55.696,89.276L56.256,101.167L56.817,131.023L57.377,369.871L57.938,226.407L58.498,133.996L59.059,67.692L59.62,340.661L60.18,370L60.741,272.419L61.301,145.886L61.862,41.713L62.422,136.322L62.983,222.4L63.544,109.697L64.104,108.663L64.665,162.818L65.225,70.794L65.786,110.602L66.346,163.076L66.907,108.792L67.467,69.501L68.028,87.208L68.589,128.05L69.149,116.418L69.71,107.758L70.27,370L70.831,308.091L71.391,177.939L71.952,143.301L72.513,105.561L73.073,126.241L73.634,125.336L74.194,96.385L74.755,91.861L75.315,181.171L75.876,126.629L76.436,61.747L76.997,276.167L77.558,135.417L78.118,135.288L78.679,117.323L79.239,81.004L79.8,102.976L80.36,95.997L80.921,111.248L81.481,106.725L82.042,155.709L82.603,76.739L83.163,370L83.724,139.036L84.284,155.192L84.845,95.351L85.405,191.381L85.966,120.166L86.527,154.158L87.087,367.544L87.648,130.247L88.208,82.555L88.769,158.811L89.329,84.106L89.89,185.306L90.45,52.182L91.011,321.274L91.572,219.04L92.132,206.374L92.693,109.439L93.253,138.39L93.814,73.637L94.374,165.532L94.935,73.637L95.495,115.772L96.056,132.315L96.617,118.744L97.177,370L97.738,117.84L98.298,86.562L98.859,148.6L99.419,108.663L99.98,145.499L100.541,176.13L101.101,104.14L101.662,245.148L102.222,99.228L102.783,207.537L103.343,201.075L103.904,57.74L104.464,171.348L105.025,97.677L105.586,64.461L106.146,139.165L106.707,114.609L107.267,174.321L107.828,95.739L108.388,166.566L108.949,370L109.51,147.308L110.07,179.749L110.631,98.84L111.191,284.568L111.752,172.899L112.312,126.499L112.873,70.148L113.433,140.199L113.994,112.928L114.555,275.262L115.115,89.535L115.676,66.787L116.236,72.603L116.797,255.1L117.357,192.544L117.918,123.01L118.478,253.29L119.039,110.214L119.6,101.167L120.16,81.651L120.721,274.616L121.281,93.541L121.842,105.561L122.402,140.458L122.963,211.285L123.524,148.859L124.084,199.653L124.645,157.518L125.205,188.925L125.766,63.556L126.326,130.764L126.887,111.248L127.447,271.126L128.008,82.426L128.569,110.343L129.129,276.038L129.69,91.086L130.25,180.137L130.811,133.996L131.371,109.309L131.932,66.529L132.492,99.874L133.053,133.22L133.614,84.365L134.174,313.519L134.735,185.048L135.295,188.021L135.856,126.37L136.416,228.863L136.977,88.501L137.538,285.99L138.098,369.871L138.659,140.846L139.219,97.806L139.78,179.62L140.34,321.662L140.901,91.473L141.461,146.016L142.022,164.11L142.583,223.176L143.143,171.348L143.704,83.848L144.264,369.871L144.825,163.076L145.385,167.47L145.946,131.928L146.507,104.915L147.067,146.791L147.628,106.725L148.188,290.772L148.749,113.962L149.309,76.481L149.87,180.783L150.43,94.963L150.991,159.586L151.552,112.928L152.112,120.812L152.673,284.697L153.233,135.676L153.794,295.812L154.354,294.649L154.915,145.24L155.475,106.466L156.036,111.377L156.597,174.45L157.157,91.861L157.718,208.7L158.278,370L158.839,124.819L159.399,81.78L159.96,156.614L160.521,97.29L161.081,87.725L161.642,129.213L162.202,156.097L162.763,106.466L163.323,128.567L163.884,148.213L164.444,102.459L165.005,107.5L165.566,160.491L166.126,175.484L166.687,84.882L167.247,331.484L167.808,95.222L168.368,349.967L168.929,370L169.489,113.187L170.05,147.566L170.611,225.502L171.171,369.871L171.732,118.486L172.292,140.07L172.853,142.655L173.413,367.932L173.974,155.968L174.535,122.88L175.095,153.641L175.656,369.871L176.216,129.084L176.777,82.297L177.337,125.724L177.898,97.677L178.458,145.757L179.019,128.438L179.58,111.248L180.14,94.705L180.701,138.519L181.261,181.558L181.822,232.74L182.382,120.812L182.943,113.445L183.504,107.112L184.064,11

In [18]:
const df = readCSV(fs.readFileSync("../output/msr.trace.csv", "utf-8")).select(
  pl.col("objective").as("hitRate"),
  pl.col("parameter").as("alpha")
);
console.log("Trace length:", df.height);
console.log(df.describe().toString());
const data = uniformSample(df, 1000)
  .toRecords()
  .map((d, i) => ({ index: i, ...d }));
const y2 = d3.scaleLog(
  d3.extent(data, (d) => d.alpha),
  d3.extent(data, (d) => 1 - d.hitRate)
);
const options = {
  x: { label: "Time", labelAnchor: "center" },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.axisY({
      axis: "left",
      color: "hsl(207, 45%, 30%)",
      label: "Miss Ratio",
      labelAnchor: "center",
      labelArrow: false,
      ticks: 5,
    }),
    Plot.axisY(y2.ticks(), {
      anchor: "right",
      color: "hsl(0, 53%, 40%)",
      label: "Alpha",
      labelAnchor: "center",
      labelArrow: false,
      y: y2,
      tickFormat: (d) => {
        let s = y2.tickFormat()(d);
        if (s === "200m") s = "0.2";
        return s;
      },
    }),
    Plot.lineY(data, { x: "index", y: (d) => 1 - d.hitRate, stroke: "steelblue" }),
    Plot.dotY(
      data,
      Plot.mapY((D) => D.map(y2), { x: "index", y: (d) => d.alpha, r: 1, stroke: "indianred" })
    ),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 84,
  marginBottom: 64,
  marginRight: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 56 },
});
d3.select(plot).select("div").style("font-size", "20px");
await saveEPS(plot, "../output/figures/adaptation_timeline_msr.eps");
Plot.plot({
  ...options,
  title: "MSR Trace - Adaptation Timeline",
});


Trace length: 16863
shape: (5, 3)
┌──────────┬──────────┬────────────┐
│ describe ┆ hitRate  ┆ alpha      │
│ ---      ┆ ---      ┆ ---        │
│ str      ┆ f64      ┆ f64        │
╞══════════╪══════════╪════════════╡
│ mean     ┆ 0.639814 ┆ 6.285147   │
│ std      ┆ 0.057661 ┆ 51.275902  │
│ min      ┆ 0.1118   ┆ 0.01       │
│ max      ┆ 0.8052   ┆ 890.215085 │
│ median   ┆ 0.6433   ┆ 0.205651   │
└──────────┴──────────┴────────────┘


MSR Trace - Adaptation Timeline 0 200 400 600 800 Time → 0.3 0.4 0.5 0.6 Miss Ratio 20m 100m 0.2 1 2 10 20 100 200 Alpha <path d="M40,20L40.561,158.578L41.121,168.765L41.682,144.142L42.242,250.988L42.803,174.994L43.363,246.078L43.924,237.944L44.484,264.839L45.045,247.764L45.606,199.031L46.166,260.881L46.727,249.596L47.287,184.081L47.848,190.823L48.408,191.995L48.969,186.353L49.53,314.671L50.09,291.221L50.651,153.888L51.211,191.922L51.772,257.95L52.332,273.853L52.893,300.674L53.453,254.652L54.014,173.235L54.575,275.758L55.135,293.053L55.696,266.598L56.256,168.398L56.817,203.794L57.377,231.055L57.938,242.268L58.498,155.061L59.059,177.192L59.62,241.022L60.18,295.618L60.741,277.443L61.301,229.296L61.862,168.472L62.422,239.996L62.983,244.686L63.544,224.68L64.104,287.923L64.665,286.091L65.225,231.861L65.786,283.892L66.346,184.887L66.907,192.435L67.467,257.291L68.028,229.883L68.589,245.639L69.149,241.168L69.71,246.005L70.27,214.933L70.831,268.283L71.391,222.848L71.952,264.619L72.513,190.31L73.073,261.907L73.634,215.153L74.194,267.697L74.755,253.773L75.315,275.465L75.876,283.892L76.436,234.646L76.997,205.113L77.558,242.927L78.118,242.048L78.679,298.622L79.239,258.243L79.8,188.405L80.36,270.408L80.921,201.082L81.481,317.822L82.042,118.859L82.603,298.622L83.163,285.798L83.724,235.086L84.284,288.436L84.845,222.481L85.405,252.161L85.966,334.824L86.527,179.684L87.087,205.846L87.648,266.378L88.208,250.036L88.769,262.274L89.329,263.153L89.89,190.603L90.45,287.85L91.011,263.007L91.572,260.075L92.132,192.875L92.693,202.182L93.253,310.274L93.814,251.501L94.374,208.924L94.935,259.636L95.495,265.498L96.056,245.052L96.617,240.216L97.177,294.518L97.738,211.269L98.298,248.35L98.859,263.446L99.419,302.653L99.98,238.677L100.541,198.737L101.101,339.148L101.662,168.472L102.222,261.541L102.783,261.688L103.343,204.234L103.904,167.079L104.464,291.587L105.025,241.388L105.586,175.946L106.146,323.539L106.707,220.722L107.267,238.164L107.828,208.924L108.388,302.286L108.949,242.121L109.51,219.916L110.07,310.201L110.631,226.072L111.191,158.725L111.752,185.546L112.312,236.332L112.873,277.004L113.433,196.686L113.994,186.133L114.555,240.729L115.115,332.845L115.676,267.55L116.236,328.888L116.797,248.057L117.357,220.796L117.918,294.592L118.478,264.546L119.039,168.472L119.6,229.663L120.16,181.736L120.721,247.544L121.281,314.378L121.842,195.66L122.402,244.539L122.963,313.499L123.524,312.033L124.084,292.979L124.645,328.668L125.205,327.862L125.766,310.494L126.326,350.067L126.887,338.268L127.447,327.862L128.008,187.598L128.569,194.927L129.129,219.33L129.69,248.936L130.25,270.775L130.811,279.202L131.371,345.304L131.932,321.633L132.492,197.638L133.053,231.788L133.614,292.686L134.174,300.015L134.735,330.794L135.295,342.006L135.856,341.933L136.416,169.131L136.977,243.66L137.538,256.485L138.098,285.211L138.659,260.149L139.219,280.228L139.78,298.256L140.34,246.152L140.901,271.288L141.461,265.278L142.022,271.654L142.583,314.451L143.143,261.614L143.704,213.907L144.264,232.374L144.825,271.141L145.385,278.689L145.946,289.975L146.507,267.33L147.067,246.078L147.628,236.698L148.188,280.668L148.749,236.771L149.309,296.717L149.87,251.428L150.43,232.155L150.991,275.245L151.552,237.577L152.112,215.299L152.673,238.31L153.233,276.637L153.794,258.83L154.354,219.916L154.915,230.542L155.475,296.277L156.036,258.683L156.597,240.069L157.157,207.312L157.718,251.721L158.278,226.878L158.839,291.734L159.399,238.457L159.96,290.488L160.521,273.486L161.081,309.835L161.642,220.942L162.202,289.462L162.763,268.943L163.323,249.523L163.884,311.813L164.444,207.092L165.005,218.451L165.566,325.444L166.126,228.93L166.687,266.524L167.247,236.991L167.808,280.595L168.368,264.472L168.929,287.41L169.489,224.826L170.05,294.738L170.611,191.263L171.171,323.465L171.732,204.673L172.292,299.722L172.853,242.854L173.413,236.185L173.974,266.891L174.535,235.232L175.095,245.345L175.656,261.614L176.216,270.482L176.777,286.824L177.337,279.422L177.898,255.312L178.458,315.624L179.019,237.504L179.58,307.123L180.14,204.16L180.701,345.

In [19]:
const df = readCSV(fs.readFileSync("../output/hm.trace.csv", "utf-8")).select(
  pl.col("objective").as("dcg"),
  pl.col("parameter").as("alpha")
);
console.log("Trace length:", df.height);
console.log(df.describe().toString());
const data = uniformSample(df, 1000)
  .toRecords()
  .map((d, i) => ({ index: i, ...d }));
const y2 = d3.scaleLog(
  d3.extent(data, (d) => d.alpha),
  d3.extent(data, (d) => d.dcg * 10000)
);
const options = {
  x: { label: "Time", labelAnchor: "center" },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.axisY({
      axis: "left",
      color: "hsl(207, 45%, 30%)",
      label: "DCG (sum over last 10k queries)",
      labelAnchor: "center",
      labelArrow: false,
      ticks: 5,
    }),
    Plot.axisY(y2.ticks(), {
      anchor: "right",
      color: "hsl(0, 53%, 40%)",
      label: "Alpha",
      labelAnchor: "center",
      labelArrow: false,
      y: y2,
      tickFormat: (d) => {
        let s = y2.tickFormat()(d);
        if (s === "200m") s = "0.2";
        return s;
      },
    }),
    Plot.lineY(data, { x: "index", y: (d) => d.dcg * 10000, stroke: "steelblue" }),
    Plot.dotY(
      data,
      Plot.mapY((D) => D.map(y2), { x: "index", y: (d) => d.alpha, r: 1, stroke: "indianred" })
    ),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 64,
  marginRight: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 56 },
});
d3.select(plot).select("div").style("font-size", "20px");
await saveEPS(plot, "../output/figures/adaptation_timeline_hm.eps");
Plot.plot({
  ...options,
  title: "H&M Trace - Adaptation Timeline",
});


Trace length: 3178
shape: (5, 3)
┌──────────┬──────────┬────────────┐
│ describe ┆ dcg      ┆ alpha      │
│ ---      ┆ ---      ┆ ---        │
│ str      ┆ f64      ┆ f64        │
╞══════════╪══════════╪════════════╡
│ mean     ┆ 0.011902 ┆ 3.292773   │
│ std      ┆ 0.005175 ┆ 25.627169  │
│ min      ┆ 0.004232 ┆ 0.01       │
│ max      ┆ 0.06587  ┆ 890.215085 │
│ median   ┆ 0.010614 ┆ 0.035938   │
└──────────┴──────────┴────────────┘


H&M Trace - Adaptation Timeline 0 200 400 600 800 Time → 100 200 300 400 500 600 DCG (sum over last 10k queries) 20m 100m 0.2 1 2 10 20 100 200 Alpha <path d="M40,20L40.561,96.739L41.121,166.451L41.682,247.322L42.242,154.107L42.803,175.75L43.363,285.817L43.924,306.431L44.484,321.794L45.045,316.309L45.606,312.808L46.166,311.262L46.727,318.891L47.287,320.139L47.848,329.394L48.408,317.876L48.969,323.745L49.53,324.042L50.09,324.58L50.651,322.965L51.211,332.297L51.772,322.811L52.332,311.497L52.893,308.084L53.453,299.362L54.014,300.994L54.575,310.825L55.135,311.091L55.696,301.032L56.256,319.875L56.817,322.124L57.377,320.319L57.938,308.089L58.498,300.159L59.059,317.029L59.62,312.481L60.18,303.908L60.741,320.984L61.301,320.976L61.862,318.844L62.422,317.632L62.983,311.951L63.544,312.252L64.104,295.709L64.665,255.12L65.225,308.467L65.786,310.966L66.346,321.047L66.907,327.037L67.467,338.003L68.028,334.692L68.589,336.457L69.149,333.372L69.71,328.471L70.27,329.425L70.831,303.207L71.391,306.37L71.952,333.82L72.513,342.611L73.073,330.554L73.634,324.314L74.194,319.051L74.755,332.567L75.315,334.307L75.876,338.909L76.436,310.023L76.997,289.358L77.558,344.652L78.118,322.404L78.679,334.648L79.239,323.974L79.8,329.501L80.36,321.787L80.921,330.096L81.481,330.504L82.042,326.899L82.603,335.165L83.163,320.045L83.724,305.378L84.284,320.352L84.845,272.442L85.405,278.19L85.966,208.682L86.527,236.82L87.087,286.028L87.648,287.584L88.208,227.757L88.769,308.155L89.329,323.435L89.89,337.87L90.45,312.52L91.011,311.404L91.572,314.593L92.132,320.703L92.693,300.219L93.253,272.311L93.814,125.12L94.374,159.607L94.935,167.263L95.495,341.276L96.056,352.339L96.617,337.154L97.177,330.895L97.738,335.501L98.298,352.852L98.859,278.127L99.419,294.827L99.98,318.201L100.541,339.036L101.101,339.06L101.662,355.365L102.222,360.26L102.783,338.961L103.343,341.122L103.904,325.409L104.464,332.453L105.025,320.281L105.586,329.894L106.146,334.577L106.707,327.844L107.267,330.048L107.828,348.48L108.388,322.328L108.949,337.485L109.51,308.621L110.07,302.717L110.631,312.244L111.191,299.603L111.752,316.921L112.312,291.124L112.873,310.438L113.433,331.537L113.994,289.723L114.555,313.973L115.115,318.289L115.676,290.99L116.236,291.49L116.797,328.849L117.357,318.478L117.918,274.537L118.478,332.457L119.039,326.32L119.6,328.81L120.16,317.649L120.721,341.026L121.281,343.845L121.842,334.153L122.402,279.018L122.963,248.738L123.524,303.21L124.084,297.627L124.645,318.114L125.205,317.184L125.766,338.253L126.326,328.59L126.887,328.894L127.447,323.841L128.008,331.09L128.569,320.835L129.129,337.195L129.69,337.525L130.25,340.615L130.811,336.324L131.371,336.626L131.932,329.914L132.492,338.243L133.053,330.497L133.614,336.016L134.174,336.006L134.735,335.055L135.295,331.126L135.856,345.358L136.416,334.223L136.977,329.243L137.538,333.518L138.098,323.919L138.659,319.231L139.219,328.166L139.78,299.105L140.34,313.943L140.901,327.144L141.461,351.314L142.022,338.86L142.583,335.262L143.143,331.852L143.704,333.478L144.264,329.152L144.825,294.378L145.385,283.442L145.946,302.104L146.507,276.984L147.067,288.54L147.628,260.599L148.188,268.157L148.749,309.367L149.309,316.66L149.87,332.435L150.43,323.618L150.991,335.083L151.552,323.139L152.112,332.155L152.673,330.656L153.233,325.344L153.794,331.948L154.354,328.671L154.915,332.374L155.475,321.777L156.036,324.565L156.597,325.627L157.157,327.169L157.718,335.547L158.278,340.391L158.839,335.541L159.399,335.735L159.96,333.056L160.521,323.285L161.081,324.717L161.642,325.502L162.202,332.472L162.763,328.185L163.323,318.028L163.884,332.586L164.444,325.532L165.005,320.619L165.566,302.082L166.126,306.664L166.687,314.073L167.247,312.128L167.808,287.515L168.368,294.443L168.929,318.857L169.489,301.345L170.05,305.823L170.611,291.704L171.171,316.173L171.732,289.781L172.292,343.965L172.853,332.849L173.413,337.703L173.974,338.453L174.535,343.691L175.095,336.605L175.656,333.161L176.216,337.287L176.777,336.128L177.337,332.052L177.898,330.328L178.458,341.458L179.019,324.965L179.58,339.243L180.14,

In [20]:
const df = readCSV(fs.readFileSync("../output/synthetic.trace.csv", "utf-8")).select(
  pl.col("objective").as("dcg"),
  pl.col("parameter").as("alpha")
);
console.log("Trace length:", df.height);
console.log(df.describe().toString());
const data = uniformSample(df, 1000)
  .toRecords()
  .map((d, i) => ({ index: i, ...d }));
const y2 = d3.scaleLog(
  d3.extent(data, (d) => d.alpha),
  d3.extent(data, (d) => d.dcg * 10000)
);
const options = {
  x: { label: "Time", labelAnchor: "center" },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.axisY({
      axis: "left",
      color: "hsl(207, 45%, 30%)",
      label: "DCG (sum over last 10k queries)",
      labelAnchor: "center",
      labelArrow: false,
      ticks: 5,
    }),
    Plot.axisY(y2.ticks(), {
      anchor: "right",
      color: "hsl(0, 53%, 40%)",
      label: "Alpha",
      labelAnchor: "center",
      labelArrow: false,
      y: y2,
      tickFormat: (d) => {
        let s = y2.tickFormat()(d);
        if (s === "200m") s = "0.2";
        return s;
      },
    }),
    Plot.lineY(data, { x: "index", y: (d) => d.dcg * 10000, stroke: "steelblue" }),
    Plot.dotY(
      data,
      Plot.mapY((D) => D.map(y2), { x: "index", y: (d) => d.alpha, r: 1, stroke: "indianred" })
    ),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 108,
  marginBottom: 64,
  marginRight: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 56 },
});
d3.select(plot).select("div").style("font-size", "20px");
await saveEPS(plot, "../output/figures/adaptation_timeline_synth.eps");
Plot.plot({
  ...options,
  title: "Synthetic E-Commerce Dataset Trace - Adaptation Timeline",
});


Trace length: 3000
shape: (5, 3)
┌──────────┬──────────┬────────────┐
│ describe ┆ dcg      ┆ alpha      │
│ ---      ┆ ---      ┆ ---        │
│ str      ┆ f64      ┆ f64        │
╞══════════╪══════════╪════════════╡
│ mean     ┆ 0.057297 ┆ 179.301529 │
│ std      ┆ 0.027315 ┆ 354.34861  │
│ min      ┆ 0.005923 ┆ 0.014175   │
│ max      ┆ 0.389271 ┆ 1000.0     │
│ median   ┆ 0.047795 ┆ 17.073526  │
└──────────┴──────────┴────────────┘


Synthetic E-Commerce Dataset Trace - Adaptation Timeline 0 200 400 600 800 Time → 1,000 2,000 3,000 DCG (sum over last 10k queries) 20m 100m 0.2 1 2 10 20 100 200 1k Alpha <path d="M40,227.246L40.561,308.822L41.121,321.612L41.682,327.445L42.242,335.212L42.803,326.584L43.363,315.207L43.924,344.774L44.484,345.623L45.045,345.529L45.606,344.313L46.166,342.979L46.727,347.219L47.287,338.465L47.848,342.974L48.408,340.939L48.969,349.275L49.53,317.259L50.09,326.161L50.651,351.74L51.211,327.987L51.772,330.663L52.332,311.619L52.893,344.827L53.453,334.636L54.014,346.722L54.575,352.592L55.135,343.796L55.696,348.84L56.256,317.726L56.817,351.031L57.377,348.771L57.938,345.41L58.498,349.549L59.059,350.534L59.62,343.281L60.18,334.02L60.741,341.271L61.301,350.506L61.862,349.256L62.422,345.295L62.983,346.219L63.544,342.611L64.104,303.434L64.665,336.542L65.225,343.444L65.786,351.669L66.346,352.139L66.907,345.752L67.467,347.517L68.028,348.622L68.589,344.029L69.149,332.562L69.71,349.018L70.27,343.398L70.831,346.939L71.391,343.688L71.952,352.168L72.513,348.256L73.073,324.557L73.634,343.364L74.194,340.456L74.755,343.561L75.315,350.202L75.876,344.891L76.436,353.098L76.997,354.939L77.558,337.692L78.118,343.64L78.679,345.781L79.239,342.306L79.8,280.067L80.36,287.953L80.921,282.119L81.481,282.952L82.042,280.765L82.603,290.97L83.163,295.286L83.724,296.319L84.284,288.265L84.845,289.583L85.405,286.726L85.966,281.029L86.527,283.972L87.087,290.322L87.648,285.85L88.208,282.426L88.769,299.228L89.329,354.242L89.89,358.55L90.45,357.018L91.011,357.486L91.572,356.389L92.132,356.223L92.693,355.436L93.253,354.507L93.814,357.1L94.374,355.116L94.935,354.697L95.495,355.371L96.056,334.503L96.617,344.468L97.177,341.465L97.738,331.755L98.298,330.82L98.859,323.963L99.419,358.008L99.98,355.449L100.541,356.147L101.101,352.599L101.662,354.349L102.222,354.063L102.783,353.648L103.343,355.501L103.904,353.654L104.464,355.313L105.025,353.12L105.586,351.622L106.146,350.133L106.707,354.172L107.267,351.231L107.828,353.301L108.388,353.778L108.949,352.1L109.51,350.992L110.07,352.953L110.631,351.647L111.191,351.89L111.752,352.792L112.312,348.89L112.873,354.01L113.433,355.698L113.994,353.499L114.555,348.469L115.115,350.963L115.676,348.513L116.236,350.799L116.797,354.478L117.357,349.898L117.918,349.93L118.478,350.85L119.039,349.091L119.6,347.086L120.16,347.353L120.721,350.825L121.281,351.873L121.842,348.809L122.402,353.992L122.963,347.702L123.524,347.495L124.084,348.074L124.645,342.958L125.205,346.198L125.766,279.779L126.326,285.072L126.887,285.972L127.447,286.694L128.008,297.065L128.569,281.522L129.129,370L129.69,359.244L130.25,363.515L130.811,359.277L131.371,358.109L131.932,349.483L132.492,356.81L133.053,357.318L133.614,351.816L134.174,349.745L134.735,353.101L135.295,354.555L135.856,355.168L136.416,354.325L136.977,355.334L137.538,360.266L138.098,360.927L138.659,350.442L139.219,352.1L139.78,355.864L140.34,353.324L140.901,349.655L141.461,351.519L142.022,358L142.583,344.186L143.143,347.493L143.704,352.5L144.264,337.878L144.825,356.153L145.385,352.399L145.946,343.286L146.507,351.708L147.067,349.923L147.628,349.182L148.188,347.192L148.749,344.3L149.309,344.368L149.87,349.659L150.43,349.803L150.991,347.475L151.552,346.344L152.112,342.317L152.673,347.677L153.233,348.565L153.794,352.06L154.354,349.87L154.915,333.594L155.475,352.888L156.036,349.689L156.597,349.628L157.157,344.483L157.718,341.013L158.278,352.891L158.839,339.957L159.399,344.507L159.96,347.352L160.521,343.705L161.081,346.248L161.642,339.591L162.202,345.322L162.763,343.162L163.323,351.054L163.884,342.917L164.444,311.341L165.005,351.532L165.566,338.948L166.126,339.714L166.687,294.473L167.247,260.259L167.808,264.294L168.368,274.028L168.929,276.501L169.489,281.232L170.05,275.874L170.611,349.151L171.171,349.305L171.732,349.018L172.292,345.521L172.853,339.259L173.413,355.626L173.974,358.773L174.535,353.903L175.095,351.485L175.656,351L176.216,344.621L176.777,331.988L177.337,327.081L177.898,364.473L178.458,353.201L179.019,352.034L179.58,355.